In [67]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django
from curlyBrace import curlyBrace
import random
import datetime

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
qid = 7638

In [68]:
q = Query.objects.get(pk=qid)

db1_maybes = Doc.objects.filter(docproject__project=q.project,docproject__relevant=3)
db1_nos = Doc.objects.filter(docproject__project=q.project,docproject__relevant=2)
db1_yeses = Doc.objects.filter(docproject__project=q.project,docproject__relevant=1)
db1_maybes.count()
db1_maybes

<QuerySet [<Doc: WOS:000398996206032>, <Doc: WOS:000457435000008>, <Doc: WOS:000414483600008>, <Doc: WOS:000348569200103>, <Doc: WOS:000414346000001>, <Doc: WOS:000345323600004>, <Doc: WOS:000353894100004>, <Doc: WOS:000433385000035>, <Doc: 2-s2.0-84991574453>, <Doc: WOS:000419407500068>, <Doc: WOS:000402137900006>, <Doc: WOS:000358807800041>, <Doc: WOS:000346215400002>, <Doc: 2-s2.0-84958940538>, <Doc: WOS:000325537300003>, <Doc: WOS:000361087100032>, <Doc: WOS:000357622800007>, <Doc: WOS:000392425800007>, <Doc: WOS:000363689700001>, <Doc: WOS:000333675500014>, '...(remaining elements truncated)...']>

In [69]:
db2_maybe = Category.objects.get(pk=365)
db2_yes = Category.objects.get(pk=363)
db2_no = Category.objects.get(pk=390)
db2_maybes = Doc.objects.filter(category=db2_maybe)
db2_nos = Doc.objects.filter(category=db2_no)
db2_yeses = Doc.objects.filter(category=db2_yes)
db2_maybes.distinct().count()

340

In [70]:
for x in [("Maybes",3),("Yes",1),("No",2)]:

    db1 = set(Doc.objects.filter(
        docproject__project=q.project,docproject__relevant=x[1]
    ).values_list('pk',flat=True))
    
    print(f"DB1 {x[0]}: {len(db1)}")

    dy = db1 & set(db2_yeses.values_list('pk',flat=True))
    print(f"\t- DB2 yes: {len(dy)}")

    dn = db1 & set(db2_nos.values_list('pk',flat=True))
    print(f"\t- DB2 no: {len(dn)}")

    dm = db1 & set(db2_maybes.values_list('pk',flat=True))
    print(f"\t- DB2 maybe: {len(dm)}")



DB1 Maybes: 238
	- DB2 yes: 29
	- DB2 no: 102
	- DB2 maybe: 93
DB1 Yes: 2580
	- DB2 yes: 1502
	- DB2 no: 953
	- DB2 maybe: 244
DB1 No: 4225
	- DB2 yes: 13
	- DB2 no: 58
	- DB2 maybe: 2


In [44]:
for d in list(dy)[:1]:
    doc = Doc.objects.get(pk=d)
    rs = DocOwnership.objects.filter(doc=doc,query=q)
    print(list(rs.values('user__username','relevant')))
    cs = doc.docusercat_set.filter(category__project=q.project)
    print(cs.values('user__username','category__name'))

[{'user__username': 'ee18ivc@leeds.ac.uk', 'relevant': 2}]
<QuerySet [{'user__username': 'ee18ivc@leeds.ac.uk', 'category__name': 'No response focus'}, {'user__username': 'ee18ivc@leeds.ac.uk', 'category__name': 'Asia & Australasia'}, {'user__username': 'ee18ivc@leeds.ac.uk', 'category__name': 'Yes'}, {'user__username': 'ee18ivc@leeds.ac.uk', 'category__name': 'Cities, settlements & key infrastructure'}, {'user__username': 'ee18ivc@leeds.ac.uk', 'category__name': 'Asia'}]>


In [71]:
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [73]:
db1m_tag, created = Tag.objects.get_or_create(
    title="db1_maybe",
    query=q,
    text="Manually assigned"
)

db1no_db2m_tag, created = Tag.objects.get_or_create(
    title="db1_no_db2_yesmaybe",
    query=q,
    text="Manually assigned"
)

db1y_db2m_tag, created = Tag.objects.get_or_create(
    title="db1_yes_db2_maybe",
    query=q,
    text="Manually assigned"
)

<Tag: db1_maybe>

In [75]:
assigned = set(DocOwnership.objects.filter(tag_id=4460).values_list('doc__id',flat=True))
print(len(assigned))
db1_maybes = Doc.objects.filter(docproject__project=q.project,docproject__relevant=3)
for d in db1_maybes:
    if d.id not in assigned:
        d.tag.add(db1m_tag)
        assigned.add(d.id)

        
db2_maybe = Category.objects.get(pk=365)
db2_yes = Category.objects.get(pk=363)
db1nos_db2_maybes = Doc.objects.filter(
    docproject__project=q.project,docproject__relevant=2,
    category__in=[db2_maybe,db2_yes]
)
for d in db1nos_db2_maybes:
    if d.id not in assigned:
        d.tag.add(db1no_db2m_tag)
        assigned.add(d.id)
        
db1yes_db2_maybes = Doc.objects.filter(
    docproject__project=q.project,docproject__relevant=1,
    category=db2_maybe
)

for d in db1yes_db2_maybes:
    if d.id not in assigned:
        d.tag.add(db1y_db2m_tag)
        assigned.add(d.id)
        
db1m_tag.update_tag()
db1no_db2m_tag.update_tag()
db1y_db2m_tag.update_tag()


85
